# Accessing UK Model Surface data from Microsoft Planetary Computer

This example notebook provides a walkthrough accessing the [Met Office UK Near Surface collection](https://planetarycomputer.microsoft.com/dataset/met-office-uk-deterministic-near-surface) on Microsoft Planetary Computer. This notebook outputs an image of UK surface temperatures across a forecast period.

First, import required libraries and set-up the pystac client to access the Planetary Computer STAC API.

In [1]:
import fsspec
import matplotlib.pyplot as plt
from pystac_client import Client
import planetary_computer
import xarray as xr

catalog = Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace,
)

Define collection and assets to retrieve and construct [STAC API filters](https://github.com/stac-api-extensions/filter) for efficient query performance against Planetary Computer API

In [6]:
collections = ["met-office-uk-deterministic-near-surface"]
asset_id = "temperature_at_surface"
forecast_extension_filters = {
    "op": "and",
    "args": [
        {
            "op": "=",
            "args": [
                {"property": "forecast:reference_datetime"},
                "2026-01-30T09:00:00Z",
            ],
        },
        {"op": "=", "args": [{"property": "forecast:horizon"}, "PT0054H00M"]},
        {"op": "=", "args": [{"property": "forecast:variable"}, "surface_temperature"]},
    ],
}

Search Planetary Computer catalog for STAC items and retrieve STAC Asset URL

In [ ]:
search = catalog.search(
    collections=collections, filter_lang="cql2-json", filter=forecast_extension_filters
)

items = search.item_collection()
print (f"Item Dictionary - {items.items[0].assets}")

asset_url = items.items[0].assets[asset_id].href
print(f"URL for specific NetCDF - {asset_url}")

Example usage: Open and inspect NetCDF data

In [ ]:
example_netcdf = xr.open_dataset(fsspec.open(asset_url, expand=True).open(), decode_timedelta=True)
example_netcdf

Plot surface temperatures on a map

In [ ]:
plt.figure(figsize=(10, 5))
example_netcdf["surface_temperature"].plot()